In [7]:
import ibis
from ibis import _
import lonboard
from lonboard.basemap import CartoBasemap # to choose color of basemap

con = ibis.get_backend()

In [2]:
# look into type infrastructure
url = (
    "s3://overturemaps-us-west-2/release/2024-07-22.0/theme=base/type=infrastructure/*"
)
t = con.read_parquet(url, table_name="infra-usa")

# filter for USA bounding box, subtype="power", and selecting only few columns
expr = t.filter(
    _.bbox.xmin > -125.0,
    _.bbox.ymin > 24.8,
    _.bbox.xmax < -65.8,
    _.bbox.ymax < 49.2,
    _.subtype == "power",
).select(["names", "geometry", "bbox", "class", "sources", "source_tags"])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
con.to_parquet(expr, "power-infra-usa.geoparquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
ibis.options.interactive = True
usa_power_infra = con.read_parquet("power-infra-usa.geoparquet")
usa_power_infra
usa_power_infra = usa_power_infra.rename(infra_class="class")
usa_power_infra.infra_class.value_counts().order_by(
    ibis.desc("infra_class_count")
).preview(max_rows=15)
plants = usa_power_infra.filter(_.infra_class=="plant")
power_lines = usa_power_infra.filter(_.infra_class=="power_line")
minor_lines = usa_power_infra.filter(_.infra_class=="minor_line")

In [8]:
lonboard.viz(
    plants,
    scatterplot_kwargs={"get_fill_color": "red"},
    polygon_kwargs={"get_fill_color": "red"},
    map_kwargs={
        "basemap_style": CartoBasemap.Positron,
        "view_state": {"longitude": -100, "latitude": 36, "zoom": 3},
    },
)

c:\local\projects\ibis-overturemaps\.venv\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…

In [9]:
plants_CA = plants.filter(
    _.bbox.xmin.between(-118.6, -117.9), _.bbox.ymin.between(34.5, 35.3)
).select(_.names.primary, _.geometry)

lonboard.viz(
    plants_CA,
    scatterplot_kwargs={"get_fill_color": "red"},
    polygon_kwargs={"get_fill_color": "red"},
    map_kwargs={
        "basemap_style": CartoBasemap.Positron,
    },
)

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…

In [10]:
lonboard.viz([minor_lines, power_lines])

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…